In [ ]:
!pip install -q crewai

In [ ]:
!pip install -q 'crewai[tools]'

In [ ]:
!pip install python-dotenv


In [ ]:
%load_ext dotenv
%dotenv


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
api_key = os.getenv('OPENAI_API_KEY')
# Ensure the key is loaded (do not print this in public or shared notebooks)
if api_key is None:
  raise ValueError("OPENAI_API_KEY must be set as an environment variable")

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, PDFSearchTool

In [ ]:
#search_tool = SerperDevTool()
pdf_tool = PDFSearchTool()


In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
!ls -a


In [ ]:
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3, openai_api_key=api_key)


In [ ]:


# List of your PDF files

import os

# Specify the directory containing the PDF files
directory_path = '/content'  # Adjust this to your folder path

# List all PDF files in the directory
pdf_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Now pdf_files contains all PDFs in the specified directory
print(f"Found {len(pdf_files)} PDF files in the directory.")
print(pdf_files)



In [ ]:
prosecutor = Agent(
    role = "Prosecuting Attorney",
    goal = "Determine the given police report facts. Based on the police report facts, determine which strategy will be best for prosecution. Review the research and report your findings in bullet points titled with Prosecution Strategy. Always keep the integrity of the report.  The article should be in paragraph format, a brief graph summary and include hyperlinks to any sources used.",
    backstory = """A high profile prosecuter. You are an expert on criminal law.""",
    verbose=True,
    allow_delegation=False,
    search_tool = [PDFSearchTool],
    llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
)

In [ ]:
defense_attorney = Agent(
    role = "Defense Attorney at law",
    goal = "Determine the given police report facts. Based on the police report facts, Determine which strategy will be best for prosecution. Review the research and report the updated findings in bullet points titled with Defense Strategy. Always keep the integrity of the report.  The article should be in paragraph format, a brief graph summary and include hyperlinks to any sources used.",
    backstory = """A high profile defense attorney. You are an expert on criminal law.""",
    verbose=True,
    allow_delegation=False,
    search_tool = [PDFSearchTool],
    llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
)

In [ ]:
judge = Agent(
    role = "Judge",
    goal = "Determine the given police report facts. Focus on the police report facts. Ensure fairness and that the law is followed. Oversee legal proceedings in court. Interprets the law: Judges apply rules of law to determine how a trial or hearing will proceed. Making decisions: Judges must be able to weigh facts, apply the law, and make quick decisions. For example, they may decide whether to hold someone in jail until trial or set conditions for their release. Protect the legal rights of all parties involved. For example, they decide what evidence can be shown to the jury, instruct jurors on applicable laws, and consider aggravating or mitigating factors for sentencing. Facilitating negotiations: Judges may facilitate negotiations between parties. Always keep the integrity of the report and focuses on the police report facts. ",
    backstory = """Judges preside over hearings and listen to the arguments of opposing parties. Judges and hearing officers oversee legal matters in court or administrative proceedings. They may conduct pretrial hearings, facilitate negotiations between opposing parties, and issue legal decisions. Judges focus on the police report facts""",
    verbose=True,
    allow_delegation=False,
    search_tool = [PDFSearchTool],
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
)


In [ ]:
script_writer = Agent(
    role = "Script Writer for legal assisting agents in a courtroom ",
    goal = "Create a script for the prosecutor based on the output of the prosecutor. The script should be movie style and brief. The script should be in a court setting and only in the prosectuor's point of view.  The article should be in markdown format. Be specific to be said in court. Keep the integrity of the police report too. ",
    backstory = """A script writer with experience in the legal field creating film and scripts.""",
    verbose=True,
    allow_delegation=False,
    search_tool = [PDFSearchTool],
    llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
)

In [ ]:


task1 = Task(
    description="""Based on the police report facts, create a document called a complaint with the criminal charges you are bringing against the client and penal codes. Be specific to this police report. Always keep the integrity of the report.  The article should be in markdown format.""",
    expected_output='Article in paragraph format and include hyperlinks to any sources used ',
    agent = prosecutor
)
task2 = Task(
    description="""Based on the police report facts and previous output from task1, Create a brief script for the prosecutor. The script should be realistic, specific and concise. The script should be in a court setting and in the point of view of the prosecutor.  The article should be in markdown format. Be specific to be said in court,while keeping the integrity of the police report . """,
    expected_output='Concise script format and include hyperlinks to any sources used ',
    agent = judge
)
task3 = Task(
    description="""Based on the  police report facts, describe your defense strategy and whether you recommend a plea deal or trial . Be specific to this police report. Always keep the integrity of the police report.  The article should be in markdown format.""",
    expected_output='Concise paragraph format and include hyperlinks to any sources used ',
    agent = defense_attorney
)

task4 = Task(
    description="""Focus on the police report facts, inform the defendant of the charges, explain their rights, and make decisions regarding the plea and bail. Ensure that your actions and decisions are based specifically on the details in the police report. Always keep the integrity of the report. Then summarize  your research and report the findings in bullet points.""",
    expected_output='Concise paragraph format and include hyperlinks to any sources used ',
    agent = judge
)




In [ ]:
task4 = Task(
    description="""Create a brief script for the prosecutor. The script should be realistic,specific and brief. The script should be in a court setting and in the point of view of the prosecutor.  The article should be in markdown format. Be specific to be said in court. Keep the integrity of the police report too. """,
    expected_output='Concise script format',
    agent = judge
)

In [ ]:


crew = Crew(
    agents = [prosecutor, script_writer, defense_attorney, judge],
    tasks = [task1, task2, task3, task4],
    verbose = 1,
)

In [ ]:
result= crew.kickoff()
print("-")
print(result)